# Training Model for Airline Sentiment

## Convert Problem to Classificaiton

In [1]:
!pip install trl transformers accelerate peft
!pip install bitsandbytes einops wandb

In [2]:
import pandas as pd
import random
import numpy as np
from transformers import AutoModelForCausalLM
from datasets import Dataset

In [3]:
categories = [
    "Seat Comfort",
    "Cabin Staff Service",
    "Food & Beverages",
    "Ground Service",
    "Inflight Entertainment",
    "Wifi & Connectivity",
    "Value For Money",
]

In [4]:
def review2sentiment(rating):
    if rating <= 3:
        sentiment = "Negative"
    else:
        sentiment = "Positive"

    return sentiment


def sentiment_prompt_template(review, category):
    return f"""Here is a customer review of an airline experience.
Did the customer have a Positive, Negative, or Neutral experience with the specific category?
Only evaluate the sentiment for the experience for the category mentioned below.

Review:
{review}

Category:
{category}

Sentiment:"""


def formatted_train(
    question,
    answer,
) -> str:
    return (
        f"<|user|>\n{question}</s>\n<|assistant|>{answer}</s>"
    )


df = pd.read_csv("Airline_review.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df["text"] = df.apply(lambda row: row["Review_Title"] + " " + row["Review"], axis=1)


prompt_dataset = {"prompts": [], "output": [], "text": []}
for category_name in categories:
    sentiment_column = f"{category_name} Sentiment Prompt"
    sentiments = (
        df[category_name]
        .apply(lambda x: review2sentiment(x) if not np.isnan(x) else "Neutral")
        .tolist()
    )
    prompts = df.apply(
        lambda row: sentiment_prompt_template(row["text"], category_name), axis=1
    ).tolist()

    prompt_dataset["prompts"] += prompts
    prompt_dataset["output"] += sentiments
    prompt_dataset["text"] += [
        formatted_train(prompt, sentiment)
        for prompt, sentiment in zip(prompts, sentiments)
    ]

In [5]:
ds = Dataset.from_dict(prompt_dataset)
ds = ds.train_test_split(test_size=0.1)

## Create Training Procedure


https://www.kaggle.com/code/tommyadams/fine-tuning-tinyllama

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
import os

base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
peft_model_id = "sms1097/tinyllama-airline-reviews"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
lora_alpha = 32
lora_dropout = 0.05
lora_rank = 16

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05
lora_rank = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)


In [10]:
output_dir = peft_model_id
per_device_train_batch_size = 32
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 10
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 100     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine"

In [11]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to='none'
)

In [12]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    peft_config=peft_config,
    max_seq_length=1024,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

Map:   0%|          | 0/145977 [00:00<?, ? examples/s]

Map:   0%|          | 0/16220 [00:00<?, ? examples/s]

In [13]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.227200
20,1.805300
30,1.786900
40,1.761500
50,1.749100
60,1.760000
70,1.728500
80,1.752500
90,1.753900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
10,2.227200
20,1.805300
30,1.786900
40,1.761500
50,1.749100
60,1.760000
70,1.728500
80,1.752500
90,1.753900
100,1.760600


TrainOutput(global_step=100, training_loss=1.808555278778076, metrics={'train_runtime': 2033.6189, 'train_samples_per_second': 3.147, 'train_steps_per_second': 0.049, 'total_flos': 2.833583753802547e+16, 'train_loss': 1.808555278778076, 'epoch': 0.04})

In [25]:
from transformers import GenerationConfig

def chat_template(question):
    return f"<|user|>\n{question}</s>\n<|assistant|>"

def run_inference(review, categories):
    category_reviews = {}
    for category_name in categories:
        prompt = sentiment_prompt_template(review, category_name)
        formatted_prompt = chat_template(prompt)
        inputs = tokenizer([formatted_prompt], return_tensors="pt")
        generation_config = GenerationConfig(
            penalty_alpha=0.6,
            do_sample = True,
            top_k=5,
            temperature=0.3,
            repetition_penalty=1.2,
            max_new_tokens=4,
            pad_token_id=tokenizer.eos_token_id
        )
        inputs = tokenizer(formatted_prompt, return_tensors="pt")
        outputs = model.generate(**inputs, generation_config=generation_config)
        clean_text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # print(clean_text_output)
        sentiment = clean_text_output.split('<|assistant|>')[-1].split('\n')[0]
        category_reviews[category_name] = sentiment

    return category_reviews


In [30]:
review = """ I have never experienced such a disaster of an Airline. 16 hrs stuck in the airport with no solutions and no staff cared. After decided to call it a night because of no solution it took me 2hrs to get my luggage. Then supposedly my luggage was lost until I gave them cash and it suddenly appeared. I booked a flight with another airline and left the following day. Now the battle began to try and get a refund and compensation for the hotel I had to book and meals. I submitted my Refund today and already gave me a date of revision for the 21st of this month."""
run_inference(review, categories)

{'Seat Comfort': 'Negative',
 'Cabin Staff Service': 'Neutral',
 'Food & Beverages': 'Neutral',
 'Ground Service': 'Negative',
 'Inflight Entertainment': 'Neutral',
 'Wifi & Connectivity': 'Neutral',
 'Value For Money': 'Negative'}

In [27]:
output_dir

'sms1097/tinyllama-airline-reviews'